In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving b.csv to b.csv
Saving d.csv to d.csv
Saving labels.csv to labels.csv
User uploaded file "b.csv" with length 2272173 bytes
User uploaded file "d.csv" with length 23766884 bytes
User uploaded file "labels.csv" with length 5937 bytes


In [3]:
import pandas as pd
import tensorflow as tf
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Embedding, concatenate, Flatten, Dropout, Lambda, Activation, BatchNormalization, LocallyConnected1D, Reshape
from keras.utils import plot_model
import time
import itertools
import random

b = pd.read_csv('b.csv')
#Changes 'na' to 'naa' and 'nan' to 'nana'
d = pd.read_csv('d.csv', index_col=0, header=None, encoding='ISO-8859-1').T
labels = pd.read_csv('labels.csv', index_col=0, header=None).T

cols = list(b) + list(set(list(d)) - set(list(b)))
d = d.loc[:,cols]
d = d.div(d.sum(axis=1), axis=0)
d.reset_index(drop=True, inplace=True)
labels.reset_index(drop=True, inplace=True)

Using TensorFlow backend.


In [4]:
#Create the true label pairs. Left: d-array, right: b-array 
#Labels start from 1 in the original file. They indicate a specific row in b table.
d['label'] = labels['label'] - 1
d['one'] = 1
d_true = list(zip(d.index, d.label, d.one))
d = d.drop(['one', 'label'], 1)

#Create the false label pairs
a_list = [i for i in range(0,1297)] # d-array
b_list = [j for j in range(0,3039)] # b-array
d_prod = list(itertools.product(a_list, b_list))
d_false = [x+(0,) for x in d_prod if x not in d_true]
#dge.ix[:,0:-1]
#dge[dge.columns[1:-1]]
#dge.iloc[:,0:-1]
#bdtnp['label'] = 0 #bdtnp.index + 1
#dge['label'] = 0
#dge_false = pd.merge(dge, bdtnp, on='label')

#Merge the two lists. Select 10003 samples from d and 1297 from b for training.
indicies = random.sample(range(len(d_false)), 10003)
d_false1 = [d_false[i] for i in indicies]
d_list = d_true + d_false1
random.shuffle(d_list)

len(d_list) #11300

11300

In [5]:
import random
from random import shuffle
import numpy as np
import itertools

#Output of generator should be a tuple `(x, y, sample_weight)` or `(x, y)`.
class generate_array(keras.utils.Sequence):
    def __init__(self, pairs_set, batch_s):
        self.pairs = pairs_set
        self.batch_size = batch_s
    
    #Return the maximume number of batches
    #Dataset size is 11300. For a batch_size=100 we need 113 iterations of 100.
    def __len__(self):
        return int(np.ceil(len(self.pairs) / float(self.batch_size)))
    
    #Returns the batch #idx
    def __getitem__(self,idx):
        X1 = np.empty((self.batch_size, 84))
        X2 = np.empty((self.batch_size, 84))
        y = np.empty((self.batch_size), dtype=int)
        batch = 0
        for i in self.pairs[self.batch_size*idx : self.batch_size*(idx+1)]:
            try:
                b_row = b.iloc[i[1]]
                d_row_1 = d.iloc[i[0]][0:84]
            except:
                print('Exception.............', i)
                continue
            X1[batch] = b_row
            X2[batch] = d_row_1
            y[batch] = i[2]
            batch = batch + 1
        return([X1, X2], y)

myg = generate_array(d_list, 100)
print(myg.__getitem__(0)[1])

[0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0
 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]


In [6]:
#Model build
print(time.strftime("%H:%M:%S"), ' Model build')

#First input model
input_a = Input(shape=(84,))
dense_a = Dense(200, activation='softplus')(input_a)

#Second input model
input_b = Input(shape=(84,))
dense_b = Dense(200, activation='softplus')(input_b)

concat = concatenate([dense_a, dense_b])
dense_c = Dense(400, activation='softplus')(concat)
dense_d = Dense(40, activation='tanh')(concat)
dense_e = Dense(1, activation='softplus')(dense_d)
model = Model(inputs=[input_a, input_b], outputs=[dense_e])
model.compile(optimizer='adam', loss='mae', metrics=['accuracy'])
print(model.summary())

07:58:53  Model build
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 84)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 84)           0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 200)          17000       input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 200)          17000       input_2[0][0]                    
_______________________________________________________________________________________

In [8]:
print(time.strftime("%H:%M:%S"), ' Fit')
tbCallBack = keras.callbacks.TensorBoard(log_dir='.', histogram_freq=0, write_graph=True, write_images=True)
history = model.fit_generator(generate_array(d_list[0:10000], 50),
                              steps_per_epoch=200,
                              epochs=1000,
                              verbose=2,
                              validation_data=generate_array(d_list[10000:13000],50),
                              class_weight={0:1, 1:10}) #, use_multiprocessing=True, workers=8) #, callbacks=[tbCallBack])
print(time.strftime("%H:%M:%S"), ' Finish')

08:00:27  Fit
Epoch 1/1000
 - 6s - loss: 0.7787 - acc: 0.5327 - val_loss: 0.4710 - val_acc: 0.5238
Epoch 2/1000
 - 5s - loss: 0.7577 - acc: 0.5575 - val_loss: 0.3247 - val_acc: 0.7246
Epoch 3/1000
 - 5s - loss: 0.7227 - acc: 0.6144 - val_loss: 0.3213 - val_acc: 0.7138
Epoch 4/1000
 - 5s - loss: 0.7005 - acc: 0.6141 - val_loss: 0.4971 - val_acc: 0.5008
Epoch 5/1000
 - 5s - loss: 0.7020 - acc: 0.6285 - val_loss: 0.3611 - val_acc: 0.6515
Epoch 6/1000
 - 5s - loss: 0.6826 - acc: 0.6470 - val_loss: 0.2997 - val_acc: 0.7208
Epoch 7/1000
 - 5s - loss: 0.6803 - acc: 0.6574 - val_loss: 0.2880 - val_acc: 0.7346
Epoch 8/1000
 - 5s - loss: 0.6710 - acc: 0.6471 - val_loss: 0.3966 - val_acc: 0.6077
Epoch 9/1000
 - 5s - loss: 0.6652 - acc: 0.6537 - val_loss: 0.3322 - val_acc: 0.6715
Epoch 10/1000
 - 5s - loss: 0.6585 - acc: 0.6657 - val_loss: 0.2834 - val_acc: 0.7277
Epoch 11/1000
 - 5s - loss: 0.6545 - acc: 0.6695 - val_loss: 0.2990 - val_acc: 0.7023
Epoch 12/1000
 - 5s - loss: 0.6597 - acc: 0.6528 

KeyboardInterrupt: ignored

In [19]:
tf.__version__

from tensorflow.python.client import device_lib
device_lib.list_local_devices()

tf.test.gpu_device_name()

'/device:GPU:0'

In [10]:
!pip uninstall tensorflow -y

Uninstalling tensorflow-1.11.0rc1:
  Successfully uninstalled tensorflow-1.11.0rc1


In [11]:
!pip install tensorflow-gpu

    100% |████████████████████████████████| 253.3MB 124kB/s 
tcmalloc: large alloc 1073750016 bytes == 0x4f6de000 @  0x7f1a7d77d1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8


In [20]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - \ done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 12.1 GB  | Proc size: 1.7 GB
GPU RAM Free: 548MB | Used: 10891MB | Util  95% | Total 11439MB
